In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from fuzzywuzzy import fuzz


In [2]:
data = pd.read_csv('data/steam-processed.csv')
data.head()

,Unnamed: 0,user_id,game_title,ratings
0,1,151603712,The Elder Scrolls V Skyrim,6
1,3,151603712,Fallout 4,5
2,5,151603712,Spore,4
3,7,151603712,Fallout New Vegas,4
4,9,151603712,Left 4 Dead 2,4


In [3]:
empty_games = data.copy()
unique_games = empty_games.game_title.unique()
unique_games


array(['The Elder Scrolls V Skyrim', 'Fallout 4', 'Spore', ...,
       'Space Colony', 'Life is Hard', 'Executive Assault'], dtype=object)

In [4]:
response = requests.get('http://api.steampowered.com/ISteamApps/GetAppList/v0002/')
response.status_code

200

In [6]:
list_of_games = response.json()
list_of_games.keys()

dict_keys(['applist'])

In [6]:
list_of_games['applist'].keys()

dict_keys(['apps'])

In [7]:
games_df = pd.DataFrame(list_of_games['applist']['apps'])
games_df.head()

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,537770,Gal*Gun: Double Peace - 'Pheromone Z' Item
4,537780,Gal*Gun: Double Peace - 'Angel Cutting Board' ...


In [8]:
needed_games = games_df[np.in1d(games_df.name, unique_games)]
needed_games.columns = ['appid', 'game_title']

In [8]:
needed_games.head()

,appid,game_title
607,738320,Starter Pack
1809,724490,Protocol
5975,669970,Hard Reset
6281,664780,Alter Ego
7754,649360,Yesterday


In [8]:
new_data = pd.merge(data, needed_games, on=['game_title'], how='left')

In [9]:
unique_games[~np.in1d(unique_games,games_df.name)].shape

(1667,)

In [30]:
new_data.loc[new_data.game_title == 'The Elder Scrolls V Skyrim', 'game_title'] = 'The Elder Scrolls V: Skyrim'
new_data.loc[new_data.game_title == 'Fallout New Vegas', 'game_title'] = 'Fallout: New Vegas'
new_data.loc[new_data.game_title == 'Dead Island Epidemic', 'game_title'] = 'Dead Island: Epidemic'


In [ ]:
leftover = unique_games[~np.in1d(unique_games,games_df.name)]
for game in leftover:
    matches = []
    for steam_game in games_df.name.values:
        ratio = fuzz.ratio(game.lower(), steam_game.lower())
        if ratio >= 60:
            matches.append((steam_game, ratio))
    match_tuple = sorted(matches, key=lambda x: x[1])[::-1]
    try:
        match = match_tuple[0][0]
    except IndexError:
        print(f'No match for {game}')
        continue
    print(f'Replacing {game}: to {match}')
    new_data.loc[new_data.game_title == game, 'game_title'] = match


Replacing The Elder Scrolls V Skyrim: to The Elder Scrolls V: Skyrim
Replacing Fallout New Vegas: to Fallout: New Vegas
Replacing Dead Island Epidemic: to Dead Island: Epidemic
Replacing Dragon Age Origins - Ultimate Edition: to Dragon Age: Origins - Ultimate Edition
Replacing SEGA Genesis & Mega Drive Classics: to SEGA Genesis & Mega Drive Classics Workshop Tool
Replacing Marvel Heroes 2015: to Marvel Heroes Omega
Replacing Jazzpunk: to Junk
Replacing Fallout New Vegas Courier's Stash: to Fallout New Vegas: Courier's Stash
Replacing Fallout New Vegas Dead Money: to Fallout New Vegas: Dead Money
Replacing Fallout New Vegas Honest Hearts: to Fallout New Vegas: Honest Hearts
Replacing Grand Theft Auto Episodes from Liberty City: to Grand Theft Auto: Episodes from Liberty City
Replacing Hitman Absolution: to Hitman: Absolution
Replacing The Elder Scrolls V Skyrim - Dawnguard: to The Elder Scrolls V: Skyrim - Dawnguard
Replacing The Elder Scrolls V Skyrim - Dragonborn: to The Elder Scrolls

In [12]:
missed_games = new_data[new_data.appid.isna()].game_title.unique()
missed_games.shape

(1647,)

In [13]:
missed_games_df = games_df[np.in1d(games_df.name, missed_games)]
missed_games_df.columns = ['appid', 'game_title']


In [40]:
new_data[new_data.appid.isnull()].loc[:, ['appid', 'game_title']].shape

(47403, 2)

In [43]:
new_data.head()

,Unnamed: 0,user_id,game_title,ratings,appid
0,1.0,151603712.0,The Elder Scrolls V: Skyrim,6.0,NaN
1,3.0,151603712.0,Fallout 4,5.0,377160.0
2,5.0,151603712.0,Spore,4.0,17390.0
3,7.0,151603712.0,Fallout: New Vegas,4.0,NaN
4,9.0,151603712.0,Left 4 Dead 2,4.0,550.0


In [21]:

for index, row in missed_games_df.iterrows():
    game_title = row['game_title']
    print(game_title)
    new_data[new_data.game_title == game_title] = new_data[new_data.game_title == game_title].fillna(row['appid'])

Blood Runs Cold
The Elder Scrolls Online: Tamriel Unlimited Gold Edition
Attrition Nuclear Domination - War Punk Music Player
Wrath of Anna
Substance Painter 2
H1Z1: Test Server
Community College Hero: Trial by Fire
Dota 2 Player Profiles: CDEC's Journey to The Majors
DRAGON QUEST HEROES: Slime Weapons and Two Bonus Maps
Thank You: The Game 2
Warhammer: End Times - Vermintide Sigmar's Blessing
Elite Dangerous: Horizons
RWBY: Grimm Eclipse
MoW: Face Off XL
Shakedown Racing One
Oh...Sir! Prototype
CS:GO Player Profiles
CS:GO Player Profiles: n0thing - Cloud9
CS:GO Player Profiles: olofmeister - fnatic
CS:GO Player Profiles: Edward - Na'Vi
CS:GO Player Profiles: markeloff - Flipsid3
Blood and Gold: Caribbean!
Metal War Online: Retribution
Dota 2 Player Profiles: EG - Suma1L
Dota 2 Player Profiles: Na'Vi - XBOCT
Dota 2 Player Profiles: Invictus Gaming - Ferrari
Dota 2 Player Profiles: MVPHot6 - HEEN
Dota 2 Player Profiles: EHOME - ROTK
Dota 2 Player Profiles: Cloud9 - N0Tail
Dota 2 Player 

In [23]:
new_data = new_data.dropna()
new_data[new_data['appid'].isna()]


,Unnamed: 0,user_id,game_title,ratings,appid


In [14]:
missed = unique_games[~np.in1d(unique_games, games_df.name)]
missed_renamed = []

In [48]:
for game in missed:
    matches = []
    for steam_game in games_df.name.values:
        ratio = fuzz.ratio(game.lower(), steam_game.lower())
        if ratio >= 60:
            matches.append((steam_game, ratio))
    match_tuple = sorted(matches, key=lambda x: x[1])[::-1]
    try:
        match = match_tuple[0][0]
    except IndexError:
        print(f'No match for {game}')
        continue
    print(f'Replacing {game}: to {match}')
    missed_renamed.append(match)
    copy_data.loc[copy_data['game_title'] == game, 'game_title'] = match



Replacing 007 Legends: to 007™ Legends
Replacing 12 Labours of Hercules II The Cretan Bull: to 12 Labours of Hercules II: The Cretan Bull
Replacing 12 Labours of Hercules III Girl Power: to 12 Labours of Hercules III: Girl Power
Replacing 1701 A.D. Gold Edition: to 1701 A.D.: Gold Edition
Replacing 1701 A.D. Sunken Dragon: to 1701 A.D.: Sunken Dragon
Replacing 18 Wheels of Steel American Long Haul: to 18 Wheels of Steel: American Long Haul
Replacing 1953 NATO vs Warsaw Pact: to 1953: NATO vs Warsaw Pact
Replacing 7 Grand Steps, Step 1 What Ancients Begat: to 7 Grand Steps, Step 1: What Ancients Begat
Replacing 7 Wonders Ancient Alien Makeover: to 7 Wonders: Ancient Alien Makeover
Replacing 8-Bit Adventures The Forgotten Journey Remastered Edition: to 8-Bit Adventures: The Forgotten Journey Remastered Edition
Replacing 9 Clues 2 The Ward: to 9 Clues 2: The Ward
Replacing 9 Clues The Secret of Serpent Creek: to 9 Clues: The Secret of Serpent Creek
Replacing 911 First Responders: to 911: 

Replacing Aveyond 3-1 Lord of Twilight: to Aveyond 3-1: Lord of Twilight
Replacing Aveyond 3-2 Gates of Night: to Aveyond 3-2: Gates of Night
Replacing BIT.TRIP Presents... Runner2 Future Legend of Rhythm Alien: to BIT.TRIP Presents... Runner2: Future Legend of Rhythm Alien
Replacing BRAINPIPE A Plunge to Unhumanity: to BRAINPIPE: A Plunge to Unhumanity
Replacing Back to the Future Ep 1 - It's About Time: to Back to the Future: Ep 1 - It's About Time
Replacing Back to the Future Ep 2 - Get Tannen!: to Back to the Future: Ep 2 - Get Tannen!
Replacing Back to the Future Ep 3 - Citizen Brown: to Back to the Future: Ep 3 - Citizen Brown
Replacing Back to the Future Ep 4 - Double Visions: to Back to the Future: Ep 4 - Double Visions
Replacing Back to the Future Ep 5 - OUTATIME: to Back to the Future: Ep 5 - OUTATIME
Replacing Baldur's Gate Enhanced Edition: to Baldur's Gate: Enhanced Edition
Replacing Baldur's Gate II Enhanced Edition: to Baldur's Gate II: Enhanced Edition
Replacing Ballads

Replacing Call of Duty Black Ops - Multiplayer OSX: to Call of Duty: Black Ops - Multiplayer OSX
Replacing Call of Duty Black Ops - OSX: to Call of Duty: Black Ops - OSX
Replacing Call of Duty Black Ops II: to Call of Duty: Black Ops II
Replacing Call of Duty Black Ops II - Multiplayer: to Call of Duty: Black Ops II - Multiplayer
Replacing Call of Duty Black Ops II - Zombies: to Call of Duty: Black Ops II - Zombies
Replacing Call of Duty Black Ops III: to Call of Duty: Black Ops III
Replacing Call of Duty Ghosts: to Call of Duty: Ghosts
Replacing Call of Duty Ghosts - Multiplayer: to Call of Duty: Ghosts - Multiplayer
Replacing Call of Duty Modern Warfare 2: to Call of Duty: Modern Warfare 2
Replacing Call of Duty Modern Warfare 2 - Multiplayer: to Call of Duty: Modern Warfare 2 - Multiplayer
Replacing Call of Duty Modern Warfare 2 - Resurgence Pack: to Call of Duty: Modern Warfare 2 - Resurgence Pack
Replacing Call of Duty Modern Warfare 2 Stimulus Package: to Call of Duty: Modern War

Replacing DCS World: to Echoes World
Replacing DOOM 3 BFG Edition: to DOOM 3: BFG Edition
Replacing DOOM 3 Resurrection of Evil: to DOOM 3: Resurrection of Evil
Replacing DOOM II Hell on Earth: to DOOM II: Hell on Earth
Replacing DRAGON QUEST HEROES Slime Weapons and Two Bonus Maps: to DRAGON QUEST HEROES: Slime Weapons and Two Bonus Maps
Replacing DYNASTY WARRIORS 8 Xtreme Legends Complete Edition: to DYNASTY WARRIORS 8: Xtreme Legends Complete Edition
Replacing Dark Arcana The Carnival: to Dark Arcana: The Carnival
Replacing Dark Fall 1 The Journal: to Dark Fall 1: The Journal
Replacing Dark Fall 2 Lights Out: to Dark Fall 2: Lights Out
Replacing Dark Fall Lost Souls: to Dark Fall: Lost Souls
Replacing Dark Souls Prepare to Die Edition: to DARK SOULS™: Prepare To Die Edition
Replacing Darkest Dungeon: to Darkest Dungeon®
Replacing Darkest Hour A Hearts of Iron Game: to Darkest Hour: A Hearts of Iron Game
Replacing Darkest Hour Europe '44-'45: to Darkest Hour: Europe '44-'45
Replacing

Replacing Duke Nukem 3D Megaton Edition: to Duke Nukem 3D: Megaton Edition
Replacing Duke Nukem Manhattan Project: to Duke Nukem: Manhattan Project
Replacing Dungeon Nightmares II  The Memory: to Dungeon Nightmares II : The Memory
Replacing Dungeons & Dragons Chronicles of Mystara: to Dungeons & Dragons: Chronicles of Mystara
Replacing Dungeons & Dragons Daggerdale: to Dungeons & Dragons: Daggerdale
Replacing Dungeons & Dragons Online: to Dungeons & Dragons Online®
Replacing Dungeons The Eye of Draconus: to Dungeons: The Eye of Draconus
Replacing Dust An Elysian Tail: to Dust: An Elysian Tail
Replacing Dynasty Warriors 8 - Empires: to Dynasty Warriors 8 Empires
Replacing Dysfunctional Systems Learning to Manage Chaos: to Dysfunctional Systems: Learning to Manage Chaos
Replacing E.Y.E Divine Cybermancy: to E.Y.E: Divine Cybermancy
Replacing ENSLAVED Odyssey to the West Premium Edition: to ENSLAVED™: Odyssey to the West™ Premium Edition
Replacing EVGA PrecisionX 16: to EVGA Precision X1


...

Replacing Serious Sam HD The First Encounter: to Serious Sam HD: The First Encounter
Replacing Serious Sam HD The Second Encounter: to Serious Sam HD: The Second Encounter
Replacing Serious Sam HD The Second Encounter Player Models: to Serious Sam HD: The Second Encounter Player Models
Replacing Serious Sam The Random Encounter: to Serious Sam: The Random Encounter
Replacing Shadow Harvest Phantom Ops: to Shadow Harvest: Phantom Ops
Replacing Shadow Ops Red Mercury: to Shadow Ops: Red Mercury
Replacing Shadowgate MacVenture Series: to Shadowgate: MacVenture Series
Replacing Shadowgrounds Survivor: to Shadowgrounds: Survivor
Replacing Shadowrun Dragonfall: to Shadowrun: Dragonfall
Replacing Shadowrun Dragonfall - Director's Cut: to Shadowrun: Dragonfall - Director's Cut
Replacing Shadowrun Hong Kong: to Shakedown Racing One
Replacing Shadows on the Vatican - Act I Greed: to Shadows on the Vatican - Act I: Greed
Replacing Shantae Risky's Revenge - Director's Cut: to Shantae: Risky's Reve

Replacing Steel Storm Burning Retribution: to Steel Storm: Burning Retribution
Replacing Strife Veteran Edition: to Strife: Veteran Edition
Replacing Strike Suit Zero Director's Cut: to Strike Suit Zero: Director's Cut
Replacing Strong Bad Episode 1 Homestar Ruiner: to Strong Bad Episode 1: Homestar Ruiner
Replacing Strong Bad Episode 2 Strong Badia the Free: to Strong Bad Episode 2: Strong Badia the Free
Replacing Strong Bad Episode 3 Baddest of the Bands: to Strong Bad Episode 3: Baddest of the Bands
Replacing Strong Bad Episode 4 Dangeresque 3: to Strong Bad Episode 4: Dangeresque 3
Replacing Strong Bad Episode 5 8-Bit Is Enough: to Strong Bad Episode 5: 8-Bit Is Enough
Replacing Styx Master of Shadows: to Styx: Master of Shadows
Replacing Sublevel Zero: to Sublevel Zero Redux
Replacing Substance Painter: to Substance Painter 2
Replacing Sugar Cube Bittersweet Factory: to Sugar Cube: Bittersweet Factory
Replacing SunAge Battle for Elysium: to SunAge: Battle for Elysium
Replacing Sun

Replacing The Elder Scrolls V Skyrim - Hearthfire: to The Elder Scrolls V: Skyrim - Hearthfire
Replacing The Escapists The Walking Dead: to The Escapists: The Walking Dead
Replacing The Great Jitters Pudding Panic: to The Great Jitters: Pudding Panic
Replacing The Guild II Renaissance: to The Guild II: Renaissance
Replacing The Hat Man Shadow Ward: to The Hat Man: Shadow Ward
Replacing The Haunted Hells Reach: to The Haunted: Hells Reach
Replacing The History Channel Civil War: to The History Channel: Civil War
Replacing The Incredible Adventures of Van Helsing Final Cut: to The Incredible Adventures of Van Helsing: Final Cut
Replacing The Incredible Adventures of Van Helsing Thaumaturge: to The Incredible Adventures of Van Helsing: Thaumaturge
Replacing The Journey Down Chapter One: to The Journey Down: Chapter One
Replacing The Journey Down Chapter Two: to The Journey Down: Chapter Two
Replacing The Joylancer Legendary Motor Knight: to The Joylancer: Legendary Motor Knight
No match f

Replacing Tom Clancy's Ghost Recon Phantoms - NA Support Starter Pack: to Tom Clancy's Ghost Recon Phantoms - NA: Support Starter Pack
Replacing Tom Clancy's Ghost Recon Phantoms - NA The Thrill of the Surprise: to Tom Clancy's Ghost Recon Phantoms - NA: The Thrill of the Surprise
Replacing Tom Clancy's Ghost Recon Phantoms - NA WAR Madness pack (Assault): to Tom Clancy's Ghost Recon Phantoms - NA: WAR Madness pack (Assault)
Replacing Tom Clancy's Ghost Recon Phantoms - NA WAR Madness pack (Recon): to Tom Clancy's Ghost Recon Phantoms - NA: WAR Madness pack (Recon)
Replacing Tom Clancy's Ghost Recon Phantoms - NA WAR Madness pack (Support): to Tom Clancy's Ghost Recon Phantoms - NA: WAR Madness pack (Support)
Replacing Tom Clancy's Rainbow Six 3 Athena Sword: to Tom Clancy's Rainbow Six 3: Athena Sword
Replacing Tom Clancy's Rainbow Six 3 Gold Edition: to Tom Clancy's Rainbow Six 3: Gold Edition
Replacing Tom Clancy's Rainbow Six Lockdown: to Tom Clancy's Rainbow Six: Lockdown
Replacin

Replacing Warhammer 40,000 Armageddon - Angels of Death: to Warhammer 40,000: Armageddon - Angels of Death
Replacing Warhammer 40,000 Armageddon - Glory of Macragge: to Warhammer 40,000: Armageddon - Glory of Macragge
Replacing Warhammer 40,000 Armageddon - Untold Battles: to Warhammer 40,000: Armageddon - Untold Battles
Replacing Warhammer 40,000 Armageddon - Vulkan's Wrath: to Warhammer 40,000: Armageddon - Vulkan's Wrath
Replacing Warhammer 40,000 Dark Nexus Arena: to Warhammer 40,000: Dark Nexus Arena
Replacing Warhammer 40,000 Dawn of War  Dark Crusade: to Warhammer 40,000: Dawn of War - Dark Crusade
Replacing Warhammer 40,000 Dawn of War  Soulstorm: to Warhammer 40,000: Dawn of War - Soulstorm
Replacing Warhammer 40,000 Dawn of War  Winter Assault: to Warhammer 40,000: Dawn of War - Winter Assault
Replacing Warhammer 40,000 Dawn of War - Game of the Year Edition: to Warhammer 40,000: Dawn of War - Game of the Year Edition
Replacing Warhammer 40,000 Dawn of War II: to Warhammer 40

In [16]:
original_games = unique_games[np.in1d(unique_games, games_df.name)]
original_games

array(['Fallout 4', 'Spore', 'Left 4 Dead 2', ..., 'Space Colony',
       'Life is Hard', 'Executive Assault'], dtype=object)

In [28]:
all_games = np.concatenate((original_games, np.array(missed_renamed)))
all_games

array(['Fallout 4', 'Spore', 'Left 4 Dead 2', ...,
       'Diner Dash: Hometown Hero', 'Kohan II: Kings of War',
       'Adventure Time:  Explore the Dungeon Because I DON’T KNOW!'],
      dtype=object)

In [29]:
needed_games = games_df[np.in1d(games_df.name, all_games)]
needed_games.columns = ['appid', 'game_title']

In [30]:
needed_games.head()

,appid,game_title
410,535120,Crawl
604,533530,Blood Runs Cold
1312,526150,The Elder Scrolls Online: Tamriel Unlimited Go...
1610,523870,Breaking Fast
3760,504350,Starter Pack


In [49]:
new_data = pd.merge(copy_data, needed_games, on=['game_title'], how='left')

In [50]:
new_data.head()

,Unnamed: 0,user_id,game_title,ratings,appid
0,1,151603712,The Elder Scrolls V: Skyrim,6,72850.0
1,3,151603712,Fallout 4,5,377160.0
2,5,151603712,Spore,4,17390.0
3,7,151603712,Fallout: New Vegas,4,22449.0
4,7,151603712,Fallout: New Vegas,4,5710.0


In [46]:
missed.sort()
missed_renamed.sort()
for idx, game in enumerate(missed):
    print(f'Change {game} to {missed_renamed[idx]}')
    new_data.loc[new_data['game_title'] == game, 'game_title'] = missed_renamed[idx]

Change 007 Legends to 007™ Legends
Change 12 Labours of Hercules II The Cretan Bull to 12 Labours of Hercules II: The Cretan Bull
Change 12 Labours of Hercules III Girl Power to 12 Labours of Hercules III: Girl Power
Change 1701 A.D. Gold Edition to 1701 A.D.: Gold Edition
Change 1701 A.D. Sunken Dragon to 1701 A.D.: Sunken Dragon
Change 18 Wheels of Steel American Long Haul to 18 Wheels of Steel: American Long Haul
Change 1953 NATO vs Warsaw Pact to 1953: NATO vs Warsaw Pact
Change 7 Grand Steps, Step 1 What Ancients Begat to 7 Grand Steps, Step 1: What Ancients Begat
Change 7 Wonders Ancient Alien Makeover to 7 Wonders: Ancient Alien Makeover
Change 8-Bit Adventures The Forgotten Journey Remastered Edition to 8-Bit Adventures: The Forgotten Journey Remastered Edition
Change 9 Clues 2 The Ward to 9 Clues 2: The Ward
Change 9 Clues The Secret of Serpent Creek to 9 Clues: The Secret of Serpent Creek
Change 911 First Responders to 911: First Responders
Change A.V.A - Alliance of Valiant 

Change Back to the Future Ep 3 - Citizen Brown to Back to the Future: Ep 3 - Citizen Brown
Change Back to the Future Ep 4 - Double Visions to Back to the Future: Ep 4 - Double Visions
Change Back to the Future Ep 5 - OUTATIME to Back to the Future: Ep 5 - OUTATIME
Change Baldur's Gate Enhanced Edition to Baldur's Gate II: Enhanced Edition
Change Baldur's Gate II Enhanced Edition to Baldur's Gate: Enhanced Edition
Change Ballads of Reemus When the Bed Bites to Ballads of Reemus: When the Bed Bites
Change Ballpoint Universe Infinite to Ballpoint Universe: Infinite
Change Batman Arkham Asylum to Batman: Arkham Asylum
Change Batman Arkham Asylum GOTY Edition to Batman: Arkham Asylum GOTY Edition
Change Batman Arkham City to Batman: Arkham City GOTY
Change Batman Arkham City GOTY to Batman: Arkham City™
Change Batman Arkham Knight to Batman: Arkham Origins -  Online Supply Drop 1
Change Batman Arkham Origins to Batman: Arkham Origins - Infinite Earths Skins Pack
Change Batman Arkham Origins

Change Call of Juarez Bound in Blood to Call of Duty: World at War
Change Call of Juarez The Cartel to Call of Juarez: Bound in Blood
Change Carmageddon 2 Carpocalypse Now to Call of Juarez: The Cartel
Change Carmageddon Reincarnation to Carmageddon 2: Carpocalypse Now
Change Carrier Command Gaea Mission to Carmageddon: Reincarnation
Change Cast of the Seven Godsends to Carrier Command: Gaea Mission
Change Castle Never Judge a Book by its Cover to Cast of the Seven Godsends Demo
Change Castlevania Lords of Shadow  Mirror of Fate HD to Castle: Never Judge a Book by its Cover
Change Castlevania Lords of Shadow - Ultimate Edition to Castlevania: Lords of Shadow - Ultimate Edition
Change Castlevania Lords of Shadow 2 to Castlevania: Lords of Shadow 2
Change Castlevania Lords of Shadow 2 - Armored Dracula Costume to Castlevania: Lords of Shadow 2 - Armored Dracula Costume
Change Castlevania Lords of Shadow 2 - Dark Dracula Costume to Castlevania: Lords of Shadow 2 - Dark Dracula Costume
Cha

Change DeathSpank Thongs Of Virtue to Deadly Premonition: The Director's Cut
Change Defender's Quest Valley of the Forgotten to Deadly Tropics
Change Defense Grid 2 A Matter of Endurance to Death Track: Resurrection
Change Defense Grid Containment DLC to Death to Spies: Moment of Truth
Change Defense Grid Resurgence Map Pack 1 to Deathspank: Thongs of Virtue
Change Defense Grid Resurgence Map Pack 2  to Defender's Quest: Valley of the Forgotten
Change Defense Grid Resurgence Map Pack 3 to Defenders of Time
Change Defense Grid Resurgence Map Pack 4 to Defense Grid 2: A Matter of Endurance
Change Defense Grid The Awakening to Defense Grid: Containment DLC
Change Defiance Arktech Revolution to Defense Grid: Resurgence Map Pack 1
Change Deja Vu MacVenture Series to Defense Grid: Resurgence Map Pack 2 
Change Delta Force Black Hawk Down to Defense Grid: Resurgence Map Pack 3
Change Delta Force Black Hawk Down - Team Sabre to Defense Grid: Resurgence Map Pack 4
Change Demise of Nations Rome 

Change Enigmatis 2 The Mists of Ravenwood to E.Y.E: Divine Cybermancy
Change Enigmatis The Ghosts of Maple Creek to ENSLAVED™: Odyssey to the West™ Premium Edition
Change Eschalon Book 1 to EVGA Precision X1
Change Eschalon Book 2 to Earth 2150: Lost Souls
Change Estranged Act I to Earth 2150: The Moon Project
Change Ethan Meteor Hunter to Earth Defense Force: Insect Armageddon
Change Ethan Meteor Hunter Deluxe Content to East India Company: Pirate Bay
Change Ether One Deluxe Edition Upgrade to East India Company: Privateer
Change Europa Universalis III Divine Wind to Easy eSports
Change Europa Universalis III Heir to the Throne to Echoes World
Change Europa Universalis IV American Dream DLC to Edna & Harvey: Harvey's New Eyes
Change Europa Universalis IV Conquest of Paradise to Edna & Harvey: The Breakout
Change Europa Universalis IV Conquistadors Unit pack  to Electronic Super Joy: Groove City
Change Europa Universalis IV National Monuments II to Elemental: Fallen Enchantress Map Pac

...

Change Nightmares from the Deep The Cursed Heart to Nandeyanen!? - The 1st Sûtra
Change Nikopol Secrets of the Immortals to Napoleon: Total War
Change Ninja Reflex Steamworks Edition to Naval War: Arctic Circle
Change Nobunaga's Ambition Souzou with Power Up Kit to Navy Field 2 : Conqueror of the Ocean
Change Nosferatu The Wrath of Malachi to NecroVisioN: Lost Company
Change Nostradamus The Last Prophecy to Necronomicon: The Dawning of Darkness
Change NotGTAV to Need for Speed: Hot Pursuit
Change Numen Contest of Heroes to Need for Speed: SHIFT
Change OCEAN CITY RACING to Need for Speed: Undercover
Change ORION Prelude to Nethergate: Resurrection
Change Oceanhorn Monster of Uncharted Seas to Never Alone: Original Soundtrack
Change Octodad Dadliest Catch to Neverwinter Nights 2: Platinum
Change Oddworld Abe's Exoddus to Nexus: The Jupiter Incident
Change Oddworld Abe's Oddysee to Night Mysteries: The Amphora Prisoner
Change Oddworld Munch's Oddysee to Nightmares from the Deep 2: The Sir

Change RPG Maker Mythos Horror Resource Pack to RIFT: Infusion Edition
Change RPG Maker Old School Modern Resource Pack to RIFT: Typhoon Edition
Change RPG Maker Royal Tiles Resource Pack to RIP 2: Strike Back
Change RPG Maker Rural Farm Tiles Resource Pack to RIP 3: The Last Hero
Change RPG Maker Samurai Resource Pack to RPG Maker MV - Classic Fantasy Music Pack
Change RPG Maker The Simple Life Music Pack to RPG Maker MV - Cover Art Characters Pack
Change RPG Maker Time Fantasy to RPG Maker MV - Essentials Set
Change RPG Maker Tyler Warren First 50 Battler Pack to RPG Maker MV - FES Resource Pack
Change RPG Maker Tyler Warren RPG Battlers  2nd 50 to RPG Maker MV - FES Resource Pack
Change RPG Maker Zombie Survival Graphic Pack to RPG Maker MV - Inspirational Vol. 4
Change RWBY Grimm Eclipse to RPG Maker MV - The Simple Life Music Pack
Change RWRZ to RPG Maker MV - Time Fantasy
Change Rabbit Hole 3D Steam Edition to RPG Maker MV - Tyler Warren RPG Battlers - 1st 50
Change Race The WTCC

Change Shadowrun Dragonfall - Director's Cut to Sam & Max 302: The Tomb of Sammun-Mak
Change Shadowrun Hong Kong to Sam & Max 303: They Stole Max's Brain!
Change Shadows on the Vatican - Act I Greed to Sam & Max 304: Beyond the Alley of the Dolls
Change Shantae Risky's Revenge - Director's Cut to Sam & Max 305: The City that Dares not Sleep
Change Shattered Horizon Arconauts to SanctuaryRPG: Black Edition
Change Shellshock 2 Blood Trails to School of Dragons: How to Train Your Dragon
Change Shelter to Schrödinger's Cat and the Raiders of the Lost Quark
Change Sherlock Holmes Crimes and Punishments to Scourge: Outbreak
Change Sherlock Holmes Nemesis to Scourge: Outbreak - Blindside
Change Ship Simulator Maritime Search and Rescue to Scourge: Outbreak Fan Pack
Change Shoot Many Robots Arena Kings to Scraps: Modular Vehicle Combat
Change Shovel Knight to Scratches: Director's Cut
Change Shower With Your Dad Simulator 2015 Do You Still Shower With Your Dad to Season of Mystery : The Cherry

Change Sword of the Stars The Pit - Mind Games to Strong Bad Episode 4: Dangeresque 3
Change Sword of the Stars The Pit - Necromancer to Strong Bad Episode 5: 8-Bit Is Enough
Change Sword of the Stars The Pit Gold DLC to Styx: Master of Shadows
Change Sword of the Stars The Pit Soundtrack to Sublevel Zero Redux
Change TDP4Team Battle to Substance Painter 2
Change TERA Accessorize Pack to Sugar Cube: Bittersweet Factory
Change TERA Explorer's Pack to SunAge: Battle for Elysium
Change TERA Starter Pack to Sunrider: Mask of Arcadius
Change TERA Steam-Powered Pack to Super Killer Hornet: Resurrection
Change TI5 - CDEC's Journey to The Majors to Super Life of Pixel
Change TI5 - EG Champions of TI5 to Superbrothers: Sword & Sworcery EP
Change TI5 - Player Profiles Cloud9 - NoTail to Supreme Commander: Forged Alliance
Change TI5 - Player Profiles Complexity - zfreek to Supreme Ruler 2020: Gold
Change TI5 - Player Profiles EG - Suma1L to Survivor Squad: Gauntlets
Change TI5 - Player Profiles E

KeyboardInterrupt: 

In [47]:
copy_data = data.copy()

In [59]:
new_data.drop(['Unnamed: 0'], axis=1).to_csv('./data/final_data.csv', index=None)

In [58]:
new_data = new_data.dropna()
new_data['appid'] = pd.to_numeric(new_data['appid'], downcast='integer')
new_data.head()

,Unnamed: 0,user_id,game_title,ratings,appid
0,1,151603712,The Elder Scrolls V: Skyrim,6,72850
1,3,151603712,Fallout 4,5,377160
2,5,151603712,Spore,4,17390
3,7,151603712,Fallout: New Vegas,4,22449
4,7,151603712,Fallout: New Vegas,4,5710
